In [1]:
#coding=utf-8
import Tushare_Token as tt
import tushare as ts
import pandas as pd
import threading
import openpyxl
import time
from Date_range_pro import Date
from datetime import  datetime
#激活tushare接口
ts.set_token(tt.token)
pro = ts.pro_api()

In [2]:
#1.1  下面是获取[沪深港通资金流向]的代码
def get_cashflow(start='20141117', end='20210105', filetype='csv'):
    time_initial  = time.process_time ()
    print('资金流向会合并成一个文件')
    date_range_cf = Date.date_range(start, end, freq="d", out_format="%Y%m%d")
    df_cf = pd.DataFrame()
    count = 0
    for i in date_range_cf:
        time_strat = time.process_time()
        #print(i)
        #print("df_cf（沪深港通资金流向）已经处理到" + i)
        df_cf = pd.concat(
            [df_cf, pro.query('moneyflow_hsgt', trade_date=i)],
            axis=0,
            ignore_index=True)
        time.sleep(0.5)
        time_end = time.process_time()
        print('正在合并{}文件，单次循环时长为{}秒'.format(i, -(time_strat - time_end)))
        count += 1
        print('目前已经合并{}个，进度为{:.3%}'.format(count, count / len(date_range_cf)))
    path_cashflow = r"H:\数据备份\沪深港股通数据\沪深港通资金流向" + str(start) + 'to' + str(end)
    if filetype == 'txt'or filetype =='csv':
        df_cf.to_csv(path_cashflow + '.txt',encoding = 'utf-8',index = False)
    elif filetype == 'excel':
        df_cf.to_excel(path_cashflow + '.xlsx',encoding = 'utf-8',index = False)

    time_final = time.process_time ()
    print("——————————————————————————————————————")
    print("总共的运行时间是:{:.5}秒".format(-(time_initial-time_final)))
    print('沪深港通资金流向写入完成，路径为{}'.format(path_cashflow))
    #df_cf.to_csv(path_cashflow+'csv')#保存为csv，不需要索引行可以改为df_cf.to_csv(path_cashflow+'csv',index=True,header=True)
    #df_cf.to_excel(path_cashflow+'xlsx')#保存为excel

In [3]:
#1.2  下面是获取[沪深股通十大成交股]的代码
def get_bigten(start='20141117', end='20210105', filetype='csv'):
    time_initial = time.process_time()
    print("十大成交股会合并成为一个文件")
    count = 0
    df_bigten_sh = pd.DataFrame()
    df_bigten_sz = pd.DataFrame()
    date_range_bigten = Date.date_range(start,
                                        end,
                                        freq="d",
                                        out_format="%Y%m%d")
    path_bigten = r"H:\数据备份\沪深港股通数据\沪深股通十大成交股_单日" + str(start) + 'to' + str(
        end)
    #下面是上证
    for i in date_range_bigten:
        time_strat = time.process_time()
        print("df_bigten_sh已经处理到" + i)
        data_get_sh = pro.hsgt_top10(trade_date=i, market_type='1')
        time.sleep(0.2)
        data_get_sh = data_get_sh.sort_values(by="rank", ascending=True)
        df_bigten_sh = pd.concat([df_bigten_sh, data_get_sh],
                                 axis=0,
                                 ignore_index=True)
        time_end = time.process_time()
        print('正在合并{}文件，单次循环时长为{}秒'.format(i, -(time_strat - time_end)))
        count += 1
        print('目前已经合并{}个，进度为{:.3%}，耗时{}s'.format(count, count / len(date_range_bigten),-(time_initial - time_end)))
    if filetype == 'txt' or filetype == 'csv':
        df_bigten_sh.to_csv(path_bigten + 'sh.' + filetype,
                            header=True,
                            encoding='utf-8',
                            index=False)
    elif filetype == 'excel':
        df_bigten_sh.to_excel(path_bigten + 'sh.' + 'xlsx',
                              header=True,
                              encoding='utf-8',
                              index=False)
        print('后缀为.xlsx')

    #下面是深证
    count = 0
    for i in date_range_bigten:
        time_strat = time.process_time()
        time.sleep(0.2)
        print("df_bigten_sz已经处理到" + i)
        data_get_sz = pro.hsgt_top10(trade_date=i, market_type='3')
        data_get_sz = data_get_sz.sort_values(by="rank", ascending=True)
        df_bigten_sz = pd.concat([df_bigten_sz, data_get_sz],
                                 axis=0,
                                 ignore_index=True)
        time_end = time.process_time()
        print('正在生成{}文件，单次循环时长为{}秒'.format(i, -(time_strat - time_end)))
        count += 1
        print("当前第二阶段处理深证数据")
        print('目前已经生成{}个，进度为{:.3%}，耗时{}s'.format(count, count / len(date_range_bigten),-(time_initial - time_end)))
    if filetype == 'txt' or filetype == 'csv':
        df_bigten_sz.to_csv(path_bigten + 'sz.' + filetype,
                            header=True,
                            encoding='utf-8',
                            index=False)
    elif filetype == 'excel':
        df_bigten_sz.to_excel(path_bigten + 'sz.' + 'xlsx',
                              header=True,
                              encoding='utf-8',
                              index=False)

    time_final = time.process_time()

    print("——————————————————————————————————————")
    print("加上总共的运行时间是:{}秒，路径为{}".format(-(time_initial - time_final),
                                        path_bigten))
    print('十大成交股写入完成')

In [4]:
#1.3  下面是获取[沪深港股通持股明细]的代码


##获取单日全部持股
#df_hk_hold= pro.hk_hold(trade_date='20190625')
def get_all_hold_oneday(start='20161205', end='20210105', filetype='csv'):
    time_initial = time.process_time()
    df_all_hold = pd.DataFrame()
    count = 0
    date_range_allhold = Date.date_range(start,
                                         end,
                                         freq="d",
                                         out_format="%Y%m%d")
    for i in date_range_allhold:
        print("df_all_hold已经更新到" + i + '，每一天都会生成一个单独的文件')
        time_strat = time.process_time()
        df_all_hold = pro.hk_hold(trade_date=i)
        time.sleep(0.125)
        #print(df_all_hold)
        if filetype == 'txt' or filetype == 'csv':
            path_all_hold = r"H:\数据备份\沪深港股通数据\深港股通持股明细_不区分交易所\沪深港股通单日全部持股_" + filetype + "版本\单日总持股明细" + str(
                i) + '.' + filetype
            df_all_hold.to_csv(path_all_hold,
                               header=True,
                               encoding='utf-8',
                               index=False)  # 保存为txt版本
        elif filetype == 'excel':
            path_all_hold = r"H:\数据备份\沪深港股通数据\深港股通持股明细_不区分交易所\沪深港股通单日全部持股_" + filetype + "版本\单日总持股明细" + str(
                i) + '.xlsx'
            df_all_hold.to_excel(path_all_hold,
                                 header=True,
                                 encoding='utf-8',
                                 index=False)  # 保存为excel版本
        time_end = time.process_time()
        print('正在合并{}文件，单次循环时长为{}秒'.format(i, -(time_strat - time_end)))
        count += 1
        print('目前已经合并{}个，进度为{:.3%}'.format(count,
                                           count / len(date_range_allhold)))
    time_final = time.process_time()
    print("——————————————————————————————————————")
    print("加上总共的运行时间是:{}秒".format(-(time_initial - time_final)))
    print('单日全部持股-按日分 全部写入完成')

In [5]:
##获取单个交易所的单日持股
###首先是SH沪股通


def get_all_hold_exchange(start='20160629',
                          end='20210105',
                          filetype='csv',
                          exchange='SH'):
    """
    exchange可以为:
    "SZ","SH","HK"
    """
    time_initial = time.process_time()
    df_hk_hold_sh = pd.DataFrame()
    date_range_sht = Date.date_range(start, end, freq="d", out_format="%Y%m%d")
    count = 0
    for i in date_range_sht:
        time_strat = time.process_time()
        time.sleep(0.2)
        print("df_all_hold(按交易所区分)已经更新到" + i)
        df_hk_hold_sh = pro.hk_hold(trade_date=i, exchange=exchange)
        print(type(filetype))
        if filetype == 'txt' or filetype == 'csv':
            path_all_hold = r"H:\数据备份\沪深港股通数据\沪深港股通区分交易所" + "\\" + exchange + "单日沪深港股通持股明细_" + filetype + "版本\\" + exchange + "单日沪深港股通持股明细" + str(
                i) + '.' + filetype
            df_hk_hold_sh.to_csv(path_all_hold,
                                 header=True,
                                 encoding='utf-8',
                                 index=False)
        elif filetype == 'excel':
            path_all_hold = r"H:\数据备份\沪深港股通数据\沪深港股通区分交易所" + "\\" + exchange + "单日沪深港股通持股明细_" + filetype + "版本\\" + exchange + "单日沪深港股通持股明细" + str(
                i) + '.xlsx'
            df_hk_hold_sh.to_excel(path_all_hold,
                                   header=True,
                                   encoding='utf-8',
                                   index=False)
        else:
            print('filetype错误')
        time_end = time.process_time()
        print('正在写入{}{}文件，单次循环时长为{:.5}秒'.format(exchange + str(i), filetype,
                                                -(time_strat - time_end)))
        count += 1
        print('目前已经写入{}个，进度为{:.3%}'.format(count, count / len(date_range_sht)))
        print("当前总耗时{:.5}".format(-(time_initial - time_end)))
    time_final = time.process_time()
    print("——————————————————————————————————————")
    print("总共的运行时间是:{:.5}秒".format(-(time_initial - time_final)))
    print('单日全部持股{}-按交易所分 全部写入完成'.format(exchange))

下面开始获取

In [6]:
def get_old_data(start, end, filetype, exchange):
    get_cashflow(start='20141117', end='20210105', filetype='excel')
    get_bigten(start='20141117', end='20210105', filetype='excel')
    #get_all_hold_exchange(start = '20160629', end = '20210105', filetype = 'csv', exchange = 'SH')
    get_all_hold_exchange(start='20160629',
                          end='20210105',
                          filetype='excel',
                          exchange='SH')
    #get_all_hold_exchange(start = '20161205', end = '20210105', filetype = 'csv', exchange = 'SZ')
    get_all_hold_exchange(start='20161205',
                          end='20210105',
                          filetype='excel',
                          exchange='SZ')
    #get_all_hold_exchange(start = '20170320', end = '20210105', filetype = 'csv', exchange = 'HK')
    get_all_hold_exchange(start='20170320',
                          end='20210105',
                          filetype='excel',
                          exchange='HK')
    get_all_hold_oneday(start='20161205', end='20210105', filetype='excel')

In [7]:
def get_update_data(start, end, filetype, exchange, key=1):
    judge = print("沪深股通资金流向和十大成交股由于需要合并成单文件，因而需要重新重头获取，速度较慢")
    print('默认不更新，更新请更改key为0')
    start = input('请以8位数字的形式输入要开始更新的日期，输入1退出')
    today = ''.join(str(datetime.now())[:10].split('-'))
    if start == 1:
        return
    else:
        print('默认更新到今日{}'.format(today))
        #get_all_hold_exchange(start = start, end = today, filetype = filetype, exchange = 'SH')
        get_all_hold_exchange(start=start,
                              end=today,
                              filetype=filetype,
                              exchange='SH')
        #get_all_hold_exchange(start = start, end = today, filetype = filetype, exchange = 'SZ')
        get_all_hold_exchange(start=start,
                              end=today,
                              filetype=filetype,
                              exchange='SZ')
        #get_all_hold_exchange(start = start, end = today, filetype = filetype, exchange = 'HK')
        get_all_hold_exchange(start=start,
                              end=today,
                              filetype=filetype,
                              exchange='HK')
        get_all_hold_oneday(start=start, end=today, filetype=filetype)
        print("沪深股通资金流向和十大成交股由于需要合并成单文件，因而需要重新重头获取，速度较慢")
        print('确认一次，更新沪深股通资金流向和十大成交股请更改key为0')
        if key == 0:
            print('沪深股通资金流向和十大成交股正在从2014117开始更新')
            get_cashflow(start='20141117', end=today, filetype=filetype)
            get_bigten(start='20141117', end=today, filetype=filetype)
        elif key == 1:
            print('确认一次，本次沪深股通资金流向和十大成交股没有进行更新')

In [8]:
#get_update_data(start='',end=''.join(str(datetime.now())[:10].split('-')),filetype='excel',exchange = '',key = 0)

In [ ]:
get_bigten(start='20141117',
           end=''.join(str(datetime.now())[:10].split('-')),
           filetype='excel')

十大成交股会合并成为一个文件
df_bigten_sh已经处理到20141117
正在合并20141117文件，单次循环时长为-0.0秒
目前已经合并1个，进度为0.042%，耗时0.03125s
df_bigten_sh已经处理到20141118
正在合并20141118文件，单次循环时长为0.015625秒
目前已经合并2个，进度为0.084%，耗时0.046875s
df_bigten_sh已经处理到20141119
正在合并20141119文件，单次循环时长为0.015625秒
目前已经合并3个，进度为0.126%，耗时0.0625s
df_bigten_sh已经处理到20141120
正在合并20141120文件，单次循环时长为0.03125秒
目前已经合并4个，进度为0.168%，耗时0.09375s
df_bigten_sh已经处理到20141121
正在合并20141121文件，单次循环时长为-0.0秒
目前已经合并5个，进度为0.210%，耗时0.09375s
df_bigten_sh已经处理到20141122
正在合并20141122文件，单次循环时长为-0.0秒
目前已经合并6个，进度为0.252%，耗时0.09375s
df_bigten_sh已经处理到20141123
正在合并20141123文件，单次循环时长为-0.0秒
目前已经合并7个，进度为0.294%，耗时0.09375s
df_bigten_sh已经处理到20141124
正在合并20141124文件，单次循环时长为-0.0秒
目前已经合并8个，进度为0.335%，耗时0.09375s
df_bigten_sh已经处理到20141125
正在合并20141125文件，单次循环时长为-0.0秒
目前已经合并9个，进度为0.377%，耗时0.09375s
df_bigten_sh已经处理到20141126
正在合并20141126文件，单次循环时长为-0.0秒
目前已经合并10个，进度为0.419%，耗时0.09375s
df_bigten_sh已经处理到20141127
正在合并20141127文件，单次循环时长为0.03125秒
目前已经合并11个，进度为0.461%，耗时0.125s
df_bigten_sh已经处理到20141128
正在合并20141128文件，单次循环时长

正在合并20150220文件，单次循环时长为0.03125秒
目前已经合并96个，进度为4.025%，耗时0.859375s
df_bigten_sh已经处理到20150221
正在合并20150221文件，单次循环时长为0.015625秒
目前已经合并97个，进度为4.067%，耗时0.875s
df_bigten_sh已经处理到20150222
正在合并20150222文件，单次循环时长为0.015625秒
目前已经合并98个，进度为4.109%，耗时0.890625s
df_bigten_sh已经处理到20150223
正在合并20150223文件，单次循环时长为-0.0秒
目前已经合并99个，进度为4.151%，耗时0.890625s
df_bigten_sh已经处理到20150224
正在合并20150224文件，单次循环时长为-0.0秒
目前已经合并100个，进度为4.193%，耗时0.890625s
df_bigten_sh已经处理到20150225
正在合并20150225文件，单次循环时长为0.03125秒
目前已经合并101个，进度为4.235%，耗时0.921875s
df_bigten_sh已经处理到20150226
正在合并20150226文件，单次循环时长为0.015625秒
目前已经合并102个，进度为4.277%，耗时0.9375s
df_bigten_sh已经处理到20150227
正在合并20150227文件，单次循环时长为0.015625秒
目前已经合并103个，进度为4.319%，耗时0.953125s
df_bigten_sh已经处理到20150228
正在合并20150228文件，单次循环时长为0.015625秒
目前已经合并104个，进度为4.361%，耗时0.96875s
df_bigten_sh已经处理到20150301
正在合并20150301文件，单次循环时长为-0.0秒
目前已经合并105个，进度为4.403%，耗时0.96875s
df_bigten_sh已经处理到20150302
正在合并20150302文件，单次循环时长为-0.0秒
目前已经合并106个，进度为4.444%，耗时0.96875s
df_bigten_sh已经处理到20150303
正在合并20150303文件，单次循环时长为-0.0秒
目

正在合并20150525文件，单次循环时长为-0.0秒
目前已经合并190个，进度为7.966%，耗时1.8125s
df_bigten_sh已经处理到20150526
正在合并20150526文件，单次循环时长为0.015625秒
目前已经合并191个，进度为8.008%，耗时1.828125s
df_bigten_sh已经处理到20150527
正在合并20150527文件，单次循环时长为0.015625秒
目前已经合并192个，进度为8.050%，耗时1.84375s
df_bigten_sh已经处理到20150528
正在合并20150528文件，单次循环时长为0.015625秒
目前已经合并193个，进度为8.092%，耗时1.859375s
df_bigten_sh已经处理到20150529
正在合并20150529文件，单次循环时长为-0.0秒
目前已经合并194个，进度为8.134%，耗时1.859375s
df_bigten_sh已经处理到20150530
正在合并20150530文件，单次循环时长为0.046875秒
目前已经合并195个，进度为8.176%，耗时1.90625s
df_bigten_sh已经处理到20150531
正在合并20150531文件，单次循环时长为-0.0秒
目前已经合并196个，进度为8.218%，耗时1.90625s
df_bigten_sh已经处理到20150601
正在合并20150601文件，单次循环时长为-0.0秒
目前已经合并197个，进度为8.260%，耗时1.90625s
df_bigten_sh已经处理到20150602
正在合并20150602文件，单次循环时长为0.03125秒
目前已经合并198个，进度为8.302%，耗时1.9375s
df_bigten_sh已经处理到20150603
正在合并20150603文件，单次循环时长为0.015625秒
目前已经合并199个，进度为8.344%，耗时1.953125s
df_bigten_sh已经处理到20150604
正在合并20150604文件，单次循环时长为-0.0秒
目前已经合并200个，进度为8.386%，耗时1.953125s
df_bigten_sh已经处理到20150605
正在合并20150605文件，单次循环时长为0.0156

正在合并20150827文件，单次循环时长为-0.0秒
目前已经合并284个，进度为11.908%，耗时2.75s
df_bigten_sh已经处理到20150828
正在合并20150828文件，单次循环时长为-0.0秒
目前已经合并285个，进度为11.950%，耗时2.75s
df_bigten_sh已经处理到20150829
正在合并20150829文件，单次循环时长为0.015625秒
目前已经合并286个，进度为11.992%，耗时2.765625s
df_bigten_sh已经处理到20150830
正在合并20150830文件，单次循环时长为0.015625秒
目前已经合并287个，进度为12.034%，耗时2.78125s
df_bigten_sh已经处理到20150831
正在合并20150831文件，单次循环时长为-0.0秒
目前已经合并288个，进度为12.075%，耗时2.78125s
df_bigten_sh已经处理到20150901
正在合并20150901文件，单次循环时长为-0.0秒
目前已经合并289个，进度为12.117%，耗时2.78125s
df_bigten_sh已经处理到20150902
正在合并20150902文件，单次循环时长为0.015625秒
目前已经合并290个，进度为12.159%，耗时2.796875s
df_bigten_sh已经处理到20150903
正在合并20150903文件，单次循环时长为-0.0秒
目前已经合并291个，进度为12.201%，耗时2.796875s
df_bigten_sh已经处理到20150904
正在合并20150904文件，单次循环时长为-0.0秒
目前已经合并292个，进度为12.243%，耗时2.796875s
df_bigten_sh已经处理到20150905
正在合并20150905文件，单次循环时长为-0.0秒
目前已经合并293个，进度为12.285%，耗时2.796875s
df_bigten_sh已经处理到20150906
正在合并20150906文件，单次循环时长为-0.0秒
目前已经合并294个，进度为12.327%，耗时2.796875s
df_bigten_sh已经处理到20150907
正在合并20150907文件，单次循环时长为0.015625秒

正在合并20151127文件，单次循环时长为0.015625秒
目前已经合并376个，进度为15.765%，耗时3.828125s
df_bigten_sh已经处理到20151128
正在合并20151128文件，单次循环时长为-0.0秒
目前已经合并377个，进度为15.807%，耗时3.828125s
df_bigten_sh已经处理到20151129
正在合并20151129文件，单次循环时长为-0.0秒
目前已经合并378个，进度为15.849%，耗时3.828125s
df_bigten_sh已经处理到20151130
正在合并20151130文件，单次循环时长为0.015625秒
目前已经合并379个，进度为15.891%，耗时3.84375s
df_bigten_sh已经处理到20151201
正在合并20151201文件，单次循环时长为0.015625秒
目前已经合并380个，进度为15.933%，耗时3.859375s
df_bigten_sh已经处理到20151202
正在合并20151202文件，单次循环时长为0.015625秒
目前已经合并381个，进度为15.975%，耗时3.875s
df_bigten_sh已经处理到20151203
正在合并20151203文件，单次循环时长为-0.0秒
目前已经合并382个，进度为16.017%，耗时3.875s
df_bigten_sh已经处理到20151204
正在合并20151204文件，单次循环时长为-0.0秒
目前已经合并383个，进度为16.059%，耗时3.875s
df_bigten_sh已经处理到20151205
正在合并20151205文件，单次循环时长为0.015625秒
目前已经合并384个，进度为16.101%，耗时3.890625s
df_bigten_sh已经处理到20151206
正在合并20151206文件，单次循环时长为0.015625秒
目前已经合并385个，进度为16.143%，耗时3.90625s
df_bigten_sh已经处理到20151207
正在合并20151207文件，单次循环时长为0.015625秒
目前已经合并386个，进度为16.184%，耗时3.921875s
df_bigten_sh已经处理到20151208
正在合并20151208文件，

...

正在合并20170305文件，单次循环时长为-0.0秒
目前已经合并840个，进度为35.220%，耗时8.1875s
df_bigten_sh已经处理到20170306
正在合并20170306文件，单次循环时长为-0.0秒
目前已经合并841个，进度为35.262%，耗时8.1875s
df_bigten_sh已经处理到20170307
正在合并20170307文件，单次循环时长为-0.0秒
目前已经合并842个，进度为35.304%，耗时8.1875s
df_bigten_sh已经处理到20170308
正在合并20170308文件，单次循环时长为-0.0秒
目前已经合并843个，进度为35.346%，耗时8.1875s
df_bigten_sh已经处理到20170309
正在合并20170309文件，单次循环时长为0.03125秒
目前已经合并844个，进度为35.388%，耗时8.21875s
df_bigten_sh已经处理到20170310
正在合并20170310文件，单次循环时长为-0.0秒
目前已经合并845个，进度为35.430%，耗时8.21875s
df_bigten_sh已经处理到20170311
正在合并20170311文件，单次循环时长为0.015625秒
目前已经合并846个，进度为35.472%，耗时8.234375s
df_bigten_sh已经处理到20170312
正在合并20170312文件，单次循环时长为0.015625秒
目前已经合并847个，进度为35.514%，耗时8.25s
df_bigten_sh已经处理到20170313
正在合并20170313文件，单次循环时长为0.015625秒
目前已经合并848个，进度为35.556%，耗时8.265625s
df_bigten_sh已经处理到20170314
正在合并20170314文件，单次循环时长为0.03125秒
目前已经合并849个，进度为35.597%，耗时8.296875s
df_bigten_sh已经处理到20170315
正在合并20170315文件，单次循环时长为-0.0秒
目前已经合并850个，进度为35.639%，耗时8.296875s
df_bigten_sh已经处理到20170316
正在合并20170316文件，单次循环时长为0.0156

正在合并20170605文件，单次循环时长为0.046875秒
目前已经合并932个，进度为39.078%，耗时9.28125s
df_bigten_sh已经处理到20170606
正在合并20170606文件，单次循环时长为0.03125秒
目前已经合并933个，进度为39.119%，耗时9.3125s
df_bigten_sh已经处理到20170607
正在合并20170607文件，单次循环时长为0.015625秒
目前已经合并934个，进度为39.161%，耗时9.328125s
df_bigten_sh已经处理到20170608
正在合并20170608文件，单次循环时长为0.015625秒
目前已经合并935个，进度为39.203%，耗时9.34375s
df_bigten_sh已经处理到20170609
正在合并20170609文件，单次循环时长为0.015625秒
目前已经合并936个，进度为39.245%，耗时9.359375s
df_bigten_sh已经处理到20170610
正在合并20170610文件，单次循环时长为-0.0秒
目前已经合并937个，进度为39.287%，耗时9.359375s
df_bigten_sh已经处理到20170611
正在合并20170611文件，单次循环时长为-0.0秒
目前已经合并938个，进度为39.329%，耗时9.359375s
df_bigten_sh已经处理到20170612
正在合并20170612文件，单次循环时长为0.015625秒
目前已经合并939个，进度为39.371%，耗时9.375s
df_bigten_sh已经处理到20170613
正在合并20170613文件，单次循环时长为0.015625秒
目前已经合并940个，进度为39.413%，耗时9.390625s
df_bigten_sh已经处理到20170614
正在合并20170614文件，单次循环时长为0.015625秒
目前已经合并941个，进度为39.455%，耗时9.40625s
df_bigten_sh已经处理到20170615
正在合并20170615文件，单次循环时长为0.015625秒
目前已经合并942个，进度为39.497%，耗时9.421875s
df_bigten_sh已经处理到20170616
正在合并2

正在合并20170905文件，单次循环时长为-0.0秒
目前已经合并1024个，进度为42.935%，耗时10.390625s
df_bigten_sh已经处理到20170906
正在合并20170906文件，单次循环时长为-0.0秒
目前已经合并1025个，进度为42.977%，耗时10.390625s
df_bigten_sh已经处理到20170907
正在合并20170907文件，单次循环时长为-0.0秒
目前已经合并1026个，进度为43.019%，耗时10.390625s
df_bigten_sh已经处理到20170908
正在合并20170908文件，单次循环时长为0.03125秒
目前已经合并1027个，进度为43.061%，耗时10.421875s
df_bigten_sh已经处理到20170909
正在合并20170909文件，单次循环时长为0.015625秒
目前已经合并1028个，进度为43.103%，耗时10.4375s
df_bigten_sh已经处理到20170910
正在合并20170910文件，单次循环时长为-0.0秒
目前已经合并1029个，进度为43.145%，耗时10.4375s
df_bigten_sh已经处理到20170911
正在合并20170911文件，单次循环时长为-0.0秒
目前已经合并1030个，进度为43.187%，耗时10.4375s
df_bigten_sh已经处理到20170912
正在合并20170912文件，单次循环时长为-0.0秒
目前已经合并1031个，进度为43.229%，耗时10.4375s
df_bigten_sh已经处理到20170913
正在合并20170913文件，单次循环时长为0.015625秒
目前已经合并1032个，进度为43.270%，耗时10.453125s
df_bigten_sh已经处理到20170914
正在合并20170914文件，单次循环时长为-0.0秒
目前已经合并1033个，进度为43.312%，耗时10.453125s
df_bigten_sh已经处理到20170915
正在合并20170915文件，单次循环时长为0.015625秒
目前已经合并1034个，进度为43.354%，耗时10.46875s
df_bigten_sh已经处理到20170916
正在合并

正在合并20171204文件，单次循环时长为-0.0秒
目前已经合并1114个，进度为46.709%，耗时11.3125s
df_bigten_sh已经处理到20171205
正在合并20171205文件，单次循环时长为0.03125秒
目前已经合并1115个，进度为46.751%，耗时11.34375s
df_bigten_sh已经处理到20171206
正在合并20171206文件，单次循环时长为0.015625秒
目前已经合并1116个，进度为46.792%，耗时11.359375s
df_bigten_sh已经处理到20171207
正在合并20171207文件，单次循环时长为0.015625秒
目前已经合并1117个，进度为46.834%，耗时11.375s
df_bigten_sh已经处理到20171208
正在合并20171208文件，单次循环时长为-0.0秒
目前已经合并1118个，进度为46.876%，耗时11.375s
df_bigten_sh已经处理到20171209
正在合并20171209文件，单次循环时长为-0.0秒
目前已经合并1119个，进度为46.918%，耗时11.375s
df_bigten_sh已经处理到20171210
正在合并20171210文件，单次循环时长为-0.0秒
目前已经合并1120个，进度为46.960%，耗时11.375s
df_bigten_sh已经处理到20171211
正在合并20171211文件，单次循环时长为-0.0秒
目前已经合并1121个，进度为47.002%，耗时11.375s
df_bigten_sh已经处理到20171212
正在合并20171212文件，单次循环时长为0.015625秒
目前已经合并1122个，进度为47.044%，耗时11.390625s
df_bigten_sh已经处理到20171213
正在合并20171213文件，单次循环时长为0.015625秒
目前已经合并1123个，进度为47.086%，耗时11.40625s
df_bigten_sh已经处理到20171214
正在合并20171214文件，单次循环时长为0.015625秒
目前已经合并1124个，进度为47.128%，耗时11.421875s
df_bigten_sh已经处理到20171215
正在合并20

正在合并20180305文件，单次循环时长为0.015625秒
目前已经合并1205个，进度为50.524%，耗时12.109375s
df_bigten_sh已经处理到20180306
正在合并20180306文件，单次循环时长为0.015625秒
目前已经合并1206个，进度为50.566%，耗时12.125s
df_bigten_sh已经处理到20180307
正在合并20180307文件，单次循环时长为0.015625秒
目前已经合并1207个，进度为50.608%，耗时12.140625s
df_bigten_sh已经处理到20180308
正在合并20180308文件，单次循环时长为-0.0秒
目前已经合并1208个，进度为50.650%，耗时12.140625s
df_bigten_sh已经处理到20180309
正在合并20180309文件，单次循环时长为-0.0秒
目前已经合并1209个，进度为50.692%，耗时12.171875s
df_bigten_sh已经处理到20180310
正在合并20180310文件，单次循环时长为-0.0秒
目前已经合并1210个，进度为50.734%，耗时12.171875s
df_bigten_sh已经处理到20180311
正在合并20180311文件，单次循环时长为0.015625秒
目前已经合并1211个，进度为50.776%，耗时12.1875s
df_bigten_sh已经处理到20180312
正在合并20180312文件，单次循环时长为0.015625秒
目前已经合并1212个，进度为50.818%，耗时12.203125s
df_bigten_sh已经处理到20180313
正在合并20180313文件，单次循环时长为-0.0秒
目前已经合并1213个，进度为50.860%，耗时12.203125s
df_bigten_sh已经处理到20180314
正在合并20180314文件，单次循环时长为0.015625秒
目前已经合并1214个，进度为50.901%，耗时12.21875s
df_bigten_sh已经处理到20180315
正在合并20180315文件，单次循环时长为0.015625秒
目前已经合并1215个，进度为50.943%，耗时12.234375s
df_bigten_sh已经